In [1]:
# Если нет нужных пакетов, они будут установлены, сеанс завершится, нужно будет стартовать снова
import importlib
packages = ['torch', 'accelerate', 'transformers']
# Проверка установки каждого пакета
for package in packages:
    try:
        importlib.import_module(package)
    except ImportError:
        if package == 'torch':
            !pip install torch
        if package == 'accelerate':
            !pip install accelerate -U
        if package == 'transformers':
            !pip install transformers[torch]
        # Перезапуск сеанса
        import os
        os.kill(os.getpid(), 9)

In [24]:
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import BertForSequenceClassification, BertTokenizer, TrainingArguments, Trainer, TrainerCallback, EarlyStoppingCallback
from transformers import LongformerTokenizer, LongformerForSequenceClassification
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
def copy_header_to_text(row):
    if pd.isna(row['text']) or row['text'] == '':
        return row['header']
    else:
        return row['text']

Загрузка датасета с 32 отзывами о Stepik

In [4]:
st_dataset = pd.read_csv('https://github.com/allseenn/transfer/raw/main/03.Tasks/sait-stepik.csv', sep='\t', encoding='utf8')
st_dataset['text'] = st_dataset.apply(copy_header_to_text, axis=1)

Загрузка датасета с 516 отзывами о GeekBrains


In [5]:
gb_dataset = pd.read_csv('https://github.com/allseenn/transfer/raw/main/03.Tasks/sait-geekbrains.csv', sep='\t', encoding='utf8')
gb_dataset['text'] = gb_dataset.apply(copy_header_to_text, axis=1)

Загрузка датасета с 516 отзывами о SkillBox

In [6]:
sb_dataset = pd.read_csv('https://github.com/allseenn/transfer/raw/main/03.Tasks/sait-skillbox-onlain-shkola.csv', sep='\t', encoding='utf8')
sb_dataset['text'] = sb_dataset.apply(copy_header_to_text, axis=1)

Загрузка датасета с 70 отзывами о SkyllFactory

In [7]:
sf_dataset = pd.read_csv('https://github.com/allseenn/transfer/raw/main/03.Tasks/sait-skillfactoryru.csv', sep='\t', encoding='utf8')
sf_dataset['text'] = sf_dataset.apply(copy_header_to_text, axis=1)

Загрузка датасета с 240 отзывами о Netologiya

In [8]:
nt_dataset = pd.read_csv('https://github.com/allseenn/transfer/raw/main/03.Tasks/sait-netologiya.csv', sep='\t', encoding='utf8')
nt_dataset['text'] = nt_dataset.apply(copy_header_to_text, axis=1)

Загрузка датасета с 364 отзывами о SkyEng

In [9]:
sk_dataset = pd.read_csv('https://github.com/allseenn/transfer/raw/main/03.Tasks/sait-onlain-shkola-angliiskogo-yazyka-skyeng-1.csv', sep='\t', encoding='utf8')
sk_dataset['text'] = sk_dataset.apply(copy_header_to_text, axis=1)

Загрузка датасета с 70 отзывами YandexPrakticum

In [10]:
ya_dataset = pd.read_csv('https://github.com/allseenn/transfer/raw/main/03.Tasks/kompyuternaya-programma-yandeks-praktikum.csv', sep='\t', encoding='utf8')
ya_dataset['text'] = ya_dataset.apply(copy_header_to_text, axis=1)

Загрузка датасета с 70 отзывами UchiRu



In [11]:
uchi_dataset = pd.read_csv('https://github.com/allseenn/transfer/raw/main/03.Tasks/sait-uchiru-uchiru-interaktivnaya-obrazovatelnaya-onlain-platforma.csv', sep='\t', encoding='utf8')
uchi_dataset['text'] = uchi_dataset.apply(copy_header_to_text, axis=1)

In [ ]:
# Загрузка предварительно обученной модели Longformer
model_name = 'allenai/longformer-base-4096'
tokenizer = LongformerTokenizer.from_pretrained(model_name)
model = LongformerForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Создание функции для предсказания сентимента
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    outputs = model(**inputs)
    predictions = torch.softmax(outputs.logits, dim=1)
    sentiment_label = 1 if predictions[0][1] > predictions[0][0] else 0  # 1 для положительного, 0 для отрицательного
    return sentiment_label

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Применение функции к столбцу текста в датасете
st_dataset['predicted'] = st_dataset['text'].apply(predict_sentiment)

Initializing global attention on CLS token...
Input ids are automatically padded from 41 to 512 to be a multiple of `config.attention_window`: 512


In [ ]:
st_dataset

,header,text,verdict,link,predicted
0,Вы все еще сомневаетесь? Начинайте учить Python!,"Вот раньше как было? Дед был сапожником, отец ...",1,https://irecommend.ru/content/vy-vse-eshche-so...,1
1,Хорошая платформа для онлайн обучения. Много и...,– это образовательная платформа и конструктор ...,1,https://irecommend.ru/content/khoroshaya-platf...,1
2,Даёшь новые нейронные связи?,"О таком ресурсе как ""Степик"" я узнала совершен...",1,https://irecommend.ru/content/daesh-novye-neir...,1
3,Самообразование - легко и бесплатно! Стоит обр...,"У нас у всех есть разное хобби, а у меня есть ...",1,https://irecommend.ru/content/samoobrazovanie-...,1
4,Заработок на фотографиях автомобилей.500 или 3...,Всем привет! Меня зовут Света и я мама в декре...,1,https://irecommend.ru/content/zarabotok-na-fot...,1
5,"Подробный отчет о моем обучении,качестве подав...","Всем доброго дня! Я знаю точно,что многие,преж...",0,https://irecommend.ru/content/rasskazyvayu-pod...,1
6,"Лучший образовательный портал для тех, кто дей...",Я проходила курсы на разных обучающих порталах...,1,https://irecommend.ru/content/luchshii-obrazov...,1
7,Возможность учиться бесплатно в удобное время.,Нашла случайно в Google Play приложение Stepik...,1,https://irecommend.ru/content/vozmozhnost-uchi...,1
8,Двоякое впечатление. Отличные преподаватели. М...,Двоякое впечатление. Мой ребёнок занимался в э...,1,https://irecommend.ru/content/dvoyakoe-vpechat...,1
9,"Издала 2 книги при помощи Ridero, расскажу о п...",Здравствуйте! В последнее время я не так часто...,1,https://irecommend.ru/content/izdala-2-knigi-p...,1


In [ ]:
# Подсчет метрик
accuracy = accuracy_score(st_dataset['verdict'], st_dataset['predicted'])
precision = precision_score(st_dataset['verdict'], st_dataset['predicted'])
recall = recall_score(st_dataset['verdict'], st_dataset['predicted'])
f1 = f1_score(st_dataset['verdict'], st_dataset['predicted'])
print(f"{accuracy = }, {precision = }, {recall = }, {f1 = }")

accuracy = 0.8125, precision = 0.8125, recall = 1.0, f1 = 0.896551724137931


In [ ]:
# Применение функции к столбцу текста в датасете
gb_dataset['predicted'] = gb_dataset['text'].apply(predict_sentiment)

In [ ]:
accuracy = accuracy_score(gb_dataset['verdict'], gb_dataset['predicted'])
precision = precision_score(gb_dataset['verdict'], gb_dataset['predicted'])
recall = recall_score(gb_dataset['verdict'], gb_dataset['predicted'])
f1 = f1_score(gb_dataset['verdict'], gb_dataset['predicted'])
print(f"{accuracy = }, {precision = }, {recall = }, {f1 = }")

accuracy = 0.7325581395348837, precision = 0.7325581395348837, recall = 1.0, f1 = 0.8456375838926175


In [ ]:
# Применение функции к столбцу текста в датасете
sb_dataset['predicted'] = sb_dataset['text'].apply(predict_sentiment)

In [ ]:
accuracy = accuracy_score(sb_dataset['verdict'], sb_dataset['predicted'])
precision = precision_score(sb_dataset['verdict'], sb_dataset['predicted'])
recall = recall_score(sb_dataset['verdict'], sb_dataset['predicted'])
f1 = f1_score(sb_dataset['verdict'], sb_dataset['predicted'])
print(f"{accuracy = }, {precision = }, {recall = }, {f1 = }")

accuracy = 0.8294573643410853, precision = 0.8294573643410853, recall = 1.0, f1 = 0.9067796610169491


In [ ]:
sb_dataset['predicted'].value_counts()

predicted
1    516
Name: count, dtype: int64

In [ ]:
gb_dataset['predicted'].value_counts()

predicted
1    516
Name: count, dtype: int64

In [ ]:
# Применение функции к столбцу текста в датасете
sf_dataset['predicted'] = sf_dataset['text'].apply(predict_sentiment)

In [ ]:
accuracy = accuracy_score(sf_dataset['verdict'], sf_dataset['predicted'])
precision = precision_score(sf_dataset['verdict'], sf_dataset['predicted'])
recall = recall_score(sf_dataset['verdict'], sf_dataset['predicted'])
f1 = f1_score(sf_dataset['verdict'], sf_dataset['predicted'])
print(f"{accuracy = }, {precision = }, {recall = }, {f1 = }")

accuracy = 0.44285714285714284, precision = 0.44285714285714284, recall = 1.0, f1 = 0.6138613861386139


In [ ]:
sf_dataset['predicted'].value_counts()

predicted
1    70
Name: count, dtype: int64

DeepPavlov RuBert

In [ ]:
# Загрузка предварительно обученной модели и токенизатора RuBERT
model_name = 'DeepPavlov/rubert-base-cased'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

def pred_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    predictions = torch.softmax(outputs.logits, dim=1)
    sentiment_label = 1 if predictions[0][1] > predictions[0][0] else 0  # 1 для положительного, 0 для отрицательного
    return sentiment_label

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
nt_dataset['pred'] = nt_dataset['header'].apply(pred_sentiment)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
nt_dataset['pred'].value_counts()

pred
1    238
0      2
Name: count, dtype: int64

In [ ]:
accuracy = accuracy_score(nt_dataset['verdict'], nt_dataset['pred'])
precision = precision_score(nt_dataset['verdict'], nt_dataset['pred'])
recall = recall_score(nt_dataset['verdict'], nt_dataset['pred'])
f1 = f1_score(nt_dataset['verdict'], nt_dataset['pred'])
print(f"{accuracy = }, {precision = }, {recall = }, {f1 = }")

accuracy = 0.7291666666666666, precision = 0.7352941176470589, recall = 0.9887005649717514, f1 = 0.8433734939759037


In [ ]:
nt_dataset.loc[nt_dataset['verdict'] != nt_dataset['pred']]

,header,text,verdict,link,predicted,pred
1,диGEEKBRAINSзайнер (ЧУДА не будет),Долго собиралась опубликовать данный отзыв и у...,0,https://irecommend.ru/content/digeekbrainszain...,1,1
4,Отвратительно,Месяц назад приобрела курс продакт менеджер на...,0,https://irecommend.ru/content/otvratitelno-5408,1,1
5,Разочаровали,Поступил на курс по системной аналитике в конц...,0,https://irecommend.ru/content/razocharovali-150,1,1
6,не рекомендую!,"Всем добрый день, училась на курсе иллюстрация...",0,https://irecommend.ru/content/ne-rekomenduyu-2964,1,1
7,Совершенно непригодная контора,"Оплатил обучение на курсе ""1C-программист: с н...",0,https://irecommend.ru/content/sovershenno-nepr...,1,1
...,...,...,...,...,...,...
206,GeekBrains - много пиара и вранье по трудоустр...,"Предполагаю, что данный отзыв получится больши...",0,https://irecommend.ru/content/geekbrains-mnogo...,1,1
218,Никому не рекомендую! Я вернула полную стоимос...,Качество курсов низкое В материалах курса мног...,0,https://irecommend.ru/content/nikomu-ne-rekome...,1,1
221,Нечестная школа: Не возвращают деньги за обуче...,"Вернули только 40% стоимости курса, хотя ни од...",0,https://irecommend.ru/content/ne-vozvrashchayu...,1,1
230,Не стоит своих денег,Училась на курсе UI UX дизайна. Курс купила за...,0,https://irecommend.ru/content/ne-stoit-svoikh-...,1,1


In [ ]:
gb_dataset['pred'] = gb_dataset['header'].apply(pred_sentiment)

In [ ]:
gb_dataset['pred'].value_counts()

pred
1    514
0      2
Name: count, dtype: int64

## Файнтюнинг

In [12]:
# Загрузка данных
train_df =  pd.concat([gb_dataset[['header', 'verdict']], ya_dataset[['header', 'verdict']], uchi_dataset[['header', 'verdict']], nt_dataset[['header', 'verdict']], sb_dataset[['header', 'verdict']], sk_dataset[['header', 'verdict']]])
train_df.columns = ['text', 'label']
train_df = train_df.dropna()
train_df = train_df.reset_index(drop=True)

eval_df = sf_dataset[['header', 'verdict']].copy()
eval_df.columns = ['text', 'label']
eval_df = eval_df.dropna()
eval_df = eval_df.reset_index(drop=True)

train_df.shape, eval_df.shape

((1776, 2), (70, 2))

In [13]:
class LogMetricsCallback(TrainerCallback):
    def __init__(self):
        self.last_step = 0

    def on_evaluate(self, args, state, control, model, eval_dataloader=None, **kwargs):
        if state.global_step > self.last_step:
            metrics = state.log_history[-1]  # Получение метрик из последней записи в истории логов
            print(metrics)  # Вывод метрик
            self.last_step = state.global_step

class EarlyStoppingCallback(TrainerCallback):
    def __init__(self, patience=1):
        self.patience = patience
        self.wait = 0
        self.best_accuracy = float('-inf')
        self.best_precision = float('-inf')

    def on_evaluate(self, args, state, control, model, eval_dataloader=None, **kwargs):
        metrics = state.log_history[-1]  # Получение метрик из последней записи в истории логов
        accuracy = metrics.get("accuracy", 0.0)
        precision = metrics.get("precision", 0.0)

        if accuracy > self.best_accuracy and precision > self.best_precision:
            self.best_accuracy = accuracy
            self.best_precision = precision
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                control.should_training_stop = True

# Определение модели и токенизатора
model_name = "DeepPavlov/rubert-base-cased"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Подготовка данных для обучения
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __getitem__(self, idx):
        text = self.data.loc[idx, "text"]
        label = self.data.loc[idx, "label"]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

    def __len__(self):
        return len(self.data)

train_dataset = CustomDataset(train_df, tokenizer, max_length=512)
eval_dataset = CustomDataset(eval_df, tokenizer, max_length=512)

# Параметры обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    do_train=True,
    do_eval=True,
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Создание Trainer с добавлением колбэков ранней остановки и логирования метрик
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[LogMetricsCallback(), EarlyStoppingCallback()]
)

# Обучение модели
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.400300,0.528434
2,0.319700,0.392309


{'eval_loss': 0.5284343957901001, 'eval_runtime': 2.1808, 'eval_samples_per_second': 32.098, 'eval_steps_per_second': 4.127, 'epoch': 1.0, 'step': 222}
{'eval_loss': 0.39230871200561523, 'eval_runtime': 2.1559, 'eval_samples_per_second': 32.47, 'eval_steps_per_second': 4.175, 'epoch': 2.0, 'step': 444}


TrainOutput(global_step=444, training_loss=0.3846369992505323, metrics={'train_runtime': 358.316, 'train_samples_per_second': 49.565, 'train_steps_per_second': 6.196, 'total_flos': 934570468638720.0, 'train_loss': 0.3846369992505323, 'epoch': 2.0})

In [14]:
# Предположим, что nt_dataset это ваш новый датасет с текстами, которые вы хотите классифицировать
# Токенизация текстов из столбца "header"
tokenized_texts = tokenizer(nt_dataset['header'].tolist(), truncation=True, padding=True)
# Получение предсказаний
input_ids = torch.tensor(tokenized_texts['input_ids'])
attention_mask = torch.tensor(tokenized_texts['attention_mask'])
# Предсказание с использованием модели
with torch.no_grad():
    # Перемещаем тензоры на устройство GPU, если они находятся на CPU
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
# Преобразование выходов модели в метки классов
predictions = torch.argmax(outputs.logits, dim=1).tolist()
# Добавление предсказаний в новый столбец 'pred' датасета
nt_dataset['pred'] = predictions
# Фильтрация строк, в которых значения в столбцах 'verdict' и 'pred' отличаются
nt_dataset.loc[nt_dataset['verdict'] != nt_dataset['pred'], ['verdict', 'pred']]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,verdict,pred
13,1,0
31,1,0
59,1,0
74,0,1
77,0,1
80,0,1
84,0,1
90,0,1


In [15]:
tokenized_texts = tokenizer(st_dataset['header'].tolist(), truncation=True, padding=True)
# Получение предсказаний
input_ids = torch.tensor(tokenized_texts['input_ids'])
attention_mask = torch.tensor(tokenized_texts['attention_mask'])
# Предсказание с использованием модели
with torch.no_grad():
    # Перемещаем тензоры на устройство GPU, если они находятся на CPU
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
# Преобразование выходов модели в метки классов
predictions = torch.argmax(outputs.logits, dim=1).tolist()
st_dataset['pred'] = predictions
# Фильтрация строк, в которых значения в столбцах 'verdict' и 'pred' отличаются
st_dataset.loc[st_dataset['verdict'] != st_dataset['pred'], ['verdict', 'pred']]

,verdict,pred
18,0,1
23,0,1
27,1,0
29,1,0


In [16]:
tokenized_texts = tokenizer(sf_dataset['header'].tolist(), truncation=True, padding=True)
# Получение предсказаний
input_ids = torch.tensor(tokenized_texts['input_ids'])
attention_mask = torch.tensor(tokenized_texts['attention_mask'])
# Предсказание с использованием модели
with torch.no_grad():
    # Перемещаем тензоры на устройство GPU, если они находятся на CPU
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
# Преобразование выходов модели в метки классов
predictions = torch.argmax(outputs.logits, dim=1).tolist()
sf_dataset['pred'] = predictions
# Фильтрация строк, в которых значения в столбцах 'verdict' и 'pred' отличаются
sf_dataset.loc[sf_dataset['verdict'] != sf_dataset['pred'], ['verdict', 'pred']]

,verdict,pred
3,1,0
24,1,0
36,0,1
37,0,1
53,0,1
58,0,1
67,0,1


## Предсказание текстового поля

In [19]:
#st_dataset['text'] = st_dataset['text'].apply(lambda x: ' '.join(word_tokenize(x)[:512]))
tokenized_texts = tokenizer(st_dataset['text'].tolist(), truncation=True, padding=True, max_length=512)
# Получение предсказаний
input_ids = torch.tensor(tokenized_texts['input_ids'])
attention_mask = torch.tensor(tokenized_texts['attention_mask'])
# Предсказание с использованием модели
with torch.no_grad():
    # Перемещаем тензоры на устройство GPU, если они находятся на CPU
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
# Преобразование выходов модели в метки классов
predictions = torch.argmax(outputs.logits, dim=1).tolist()
st_dataset['predict'] = predictions
# Фильтрация строк, в которых значения в столбцах 'verdict' и 'pred' отличаются
st_dataset.loc[st_dataset['verdict'] != st_dataset['predict'], ['verdict', 'predict']]

,verdict,predict
5,0,1
12,0,1
18,0,1
23,0,1


In [20]:
tokenized_texts = tokenizer(sf_dataset['text'].tolist(), truncation=True, padding=True, max_length=512)
# Получение предсказаний
input_ids = torch.tensor(tokenized_texts['input_ids'])
attention_mask = torch.tensor(tokenized_texts['attention_mask'])
# Предсказание с использованием модели
with torch.no_grad():
    # Перемещаем тензоры на устройство GPU, если они находятся на CPU
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
# Преобразование выходов модели в метки классов
predictions = torch.argmax(outputs.logits, dim=1).tolist()
sf_dataset['predict'] = predictions
# Фильтрация строк, в которых значения в столбцах 'verdict' и 'pred' отличаются
sf_dataset.loc[sf_dataset['verdict'] != sf_dataset['predict'], ['verdict', 'predict']]

,verdict,predict
0,0,1
2,0,1
3,1,0
7,0,1
8,1,0
11,0,1
13,0,1
15,0,1
16,0,1
17,0,1


In [40]:
# sf_dataset['text'] = sf_dataset['text'].apply(lambda x: ' '.join(word_tokenize(x)[:512]))
tokenized_texts = tokenizer(nt_dataset['text'].tolist(), truncation=True, padding=True, max_length=512)
# Получение предсказаний
input_ids = torch.tensor(tokenized_texts['input_ids'])
attention_mask = torch.tensor(tokenized_texts['attention_mask'])
# Предсказание с использованием модели
with torch.no_grad():
    # Перемещаем тензоры на устройство GPU, если они находятся на CPU
    input_ids = input_ids.to(model.device)
    attention_mask = attention_mask.to(model.device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
# Преобразование выходов модели в метки классов
predictions = torch.argmax(outputs.logits, dim=1).tolist()
nt_dataset['predict'] = predictions
# Фильтрация строк, в которых значения в столбцах 'verdict' и 'pred' отличаются
nt_dataset.loc[nt_dataset['verdict'] != nt_dataset['predict'], ['verdict', 'predict']]

,verdict,predict
1,0,1
5,0,1
10,0,1
27,0,1
47,0,1
48,0,1
54,0,1
55,0,1
58,0,1
61,0,1


## Сравение метки verdict c pred и predict

In [35]:
mask = (st_dataset['verdict'] != st_dataset['pred']) | (st_dataset['verdict'] != st_dataset['predict']) | (st_dataset['pred'] != st_dataset['predict'])
filtered_dataset = st_dataset[mask]
filtered_dataset[['verdict', 'pred', 'predict']]

,verdict,pred,predict
5,0,0,1
12,0,0,1
18,0,1,1
23,0,1,1
27,1,0,1
29,1,0,1


In [36]:
mask2 = (sf_dataset['verdict'] != sf_dataset['pred']) | (sf_dataset['verdict'] != sf_dataset['predict']) | (sf_dataset['pred'] != sf_dataset['predict'])
filtered2_dataset = sf_dataset[mask2]
filtered2_dataset[['verdict', 'pred', 'predict']]

,verdict,pred,predict
0,0,0,1
2,0,0,1
3,1,0,0
7,0,0,1
8,1,1,0
11,0,0,1
13,0,0,1
15,0,0,1
16,0,0,1
17,0,0,1


In [41]:
mask3 = (nt_dataset['verdict'] != nt_dataset['pred']) | (nt_dataset['verdict'] != nt_dataset['predict']) | (nt_dataset['pred'] != nt_dataset['predict'])
filtered3_dataset = nt_dataset[mask3]
filtered3_dataset[['verdict', 'pred', 'predict']]

,verdict,pred,predict
1,0,0,1
5,0,0,1
10,0,0,1
13,1,0,1
27,0,0,1
31,1,0,1
47,0,0,1
48,0,0,1
54,0,0,1
55,0,0,1
